In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

%load_ext autoreload

%autoreload 2

os.chdir('/content/drive/MyDrive/Dacon/HAI')

Mounted at /content/drive


# Import

In [ ]:
import os, random, numpy as np, pandas as pd
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset, ConcatDataset
from torchvision import transforms, datasets, models
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torchvision.models import convnext_large, ConvNeXt_Large_Weights
# from torchvision.models import convnextv2_base, ConvNeXt_V2_Base_Weights
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.amp import autocast, GradScaler
from sklearn.model_selection import StratifiedKFold
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from functools import partial
# import timm
from sklearn.model_selection import StratifiedShuffleSplit
from transformers import get_cosine_schedule_with_warmup


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE': 424,
    'BATCH_SIZE': 32,
    'EPOCHS': 30,
    'LEARNING_RATE': 1e-4,
    'TEMPERTURE': 1.0,
    'WEIGHT_DECAY': 0.01,
    'DROPOUT_RATE': 0.1,
    'SEED': 42
}

# Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

# CustomDataset

In [ ]:
# ✅ Custom Dataset (ImageFolder 기반)
train_root = './data/train'
test_root = './data/test'

In [ ]:
full_dataset = datasets.ImageFolder(train_root)
class_names = full_dataset.classes
num_classes = len(class_names)
targets = [sample[1] for sample in full_dataset.samples]
print(num_classes)

396


In [ ]:
# ✅ ImageFolder 기반으로 전체 개수 계산 (train)
train_dataset = datasets.ImageFolder(train_root)
num_train_images = len(train_dataset)
# 이미지 확장자 필터링 포함
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.webp')
num_test_images = len([
    fname for fname in os.listdir(test_root)
    if fname.lower().endswith(image_extensions)
])
print(f"✅ 총 클래스 수: {len(train_dataset.classes)}")
print(f"🖼️ Train 이미지 수: {num_train_images}")
print(f"🖼️ Test 이미지 수: {num_test_images}")

✅ 총 클래스 수: 396
🖼️ Train 이미지 수: 33023
🖼️ Test 이미지 수: 8258


# Data Load

In [ ]:
def one_hot(labels, num_classes):
    return F.one_hot(labels.long(), num_classes=num_classes).float()

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix_collate(batch, alpha=1.0, num_classes=10):
    images, targets = zip(*batch)
    images = torch.stack(images)
    targets = torch.tensor(targets)

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(images.size(0))

    shuffled_images = images[rand_index]
    shuffled_targets = targets[rand_index]

    bbx1, bby1, bbx2, bby2 = rand_bbox(images.size(), lam)
    images[:, :, bby1:bby2, bbx1:bbx2] = shuffled_images[:, :, bby1:bby2, bbx1:bbx2]

    targets = one_hot(targets, num_classes)
    shuffled_targets = one_hot(shuffled_targets, num_classes)
    mixed_targets = targets * lam + shuffled_targets * (1. - lam)

    return images, mixed_targets

def conditional_cutmix_collate(batch, alpha=1.0, num_classes=10, cutmix_prob=0.3):
    images, targets = zip(*batch)
    images = torch.stack(images)
    targets = torch.tensor(targets)

    if np.random.rand() < cutmix_prob:
        # CutMix 적용
        lam = np.random.beta(alpha, alpha)
        rand_index = torch.randperm(images.size(0))

        shuffled_images = images[rand_index]
        shuffled_targets = targets[rand_index]

        bbx1, bby1, bbx2, bby2 = rand_bbox(images.size(), lam)
        images[:, :, bby1:bby2, bbx1:bbx2] = shuffled_images[:, :, bby1:bby2, bbx1:bbx2]

        mixed_targets = one_hot(targets, num_classes) * lam + one_hot(shuffled_targets, num_classes) * (1 - lam)
        # hard_labels = mixed_targets.argmax(dim=1)
        return images, mixed_targets
    else:
        # 원본 데이터 그대로
        return images, one_hot(targets, num_classes)

In [ ]:
class TBHalfAugment:
    def __call__(self, img: Image.Image):
        w, h = img.size
        if random.random() > 0.5:
            img = img.crop((0, 0, w, h // 2))  # 위쪽 절반
        else:
            img = img.crop((0, h // 2, w, h))  # 아래쪽 절반
        img = img.resize((w, h))  # 다시 원래 크기로 리사이즈
        return img

class RLHalfAugment:
    def __call__(self, img: Image.Image):
        w, h = img.size
        if random.random() > 0.5:
            img = img.crop((0, 0, w // 2, h))  # 왼쪽 절반
        else:
            img = img.crop((w // 2, 0, w, h))  # 오른쪽 절반
        img = img.resize((w, h))  # 다시 원래 크기로 리사이즈
        return img

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.RandomApply([TBHalfAugment()], p=0.1),
    transforms.RandomApply([RLHalfAugment()], p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.03),
    transforms.RandomPerspective(distortion_scale=0.05, p=0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.05, value='random', inplace=False),
])
val_transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.RandomApply([TBHalfAugment()], p=0.01),
    transforms.RandomApply([RLHalfAugment()], p=0.01),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
def log_loss_manual(logits, labels, eps=1e-15):
    """
    logits: (batch_size, num_classes)
    labels: (batch_size,) 정수 인덱스
    """
    probs = F.softmax(logits, dim=1)  # (N, C)
    probs = torch.clamp(probs, eps, 1. - eps)  # log(0) 방지

    # 정답 클래스 확률만 추출
    true_probs = probs[torch.arange(len(labels)), labels]  # (N,)
    loss = -torch.log(true_probs)  # (N,)
    return loss.mean()

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, num_classes, smoothing=0.02):
        super(LabelSmoothingLoss, self).__init__()
        self.num_classes = num_classes
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, logits, target):
        # logits: (N, C), target: (N,)
        log_probs = F.log_softmax(logits, dim=-1)  # (N, C)

        # One-hot with smoothing
        true_dist = torch.zeros_like(log_probs)
        true_dist.fill_(self.smoothing / (self.num_classes - 1))
        true_dist.scatter_(1, target.unsqueeze(1), self.confidence)

        return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=1, reduction='mean'): # gamma=1 유지
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)

        focal_loss = self.alpha * (1 - pt)**self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else: # 'none'
            return focal_loss

In [ ]:
class SoftTargetFocalLoss(nn.Module):
    def __init__(self, gamma=2.0, reduction='mean'):
        """
        gamma: Focal tuning parameter (default 2.0)
        reduction: 'mean' | 'sum' | 'none'
        """
        super(SoftTargetFocalLoss, self).__init__()
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        inputs: [batch_size, num_classes] - raw logits (before softmax)
        targets: [batch_size, num_classes] - soft labels (probabilities, e.g., from CutMix or MixUp)
        """
        log_probs = F.log_softmax(inputs, dim=1)        # [B, C]
        probs = log_probs.exp()                          # [B, C]

        focal_weight = (1.0 - probs) ** self.gamma       # [B, C]
        loss = -targets * focal_weight * log_probs       # [B, C]
        loss = loss.sum(dim=1)                           # [B]

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# Train/ Validation

In [ ]:
# ✅ Stratified Holdout Split (97% train / 3% val)
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.03, random_state=CFG['SEED'])
train_idx, val_idx = next(splitter.split(np.arange(len(targets)), targets))

# ✅ Dataset 준비
train_dataset = Subset(datasets.ImageFolder(train_root, transform=train_transform), train_idx)
val_dataset = Subset(datasets.ImageFolder(train_root, transform=val_transform), val_idx)

train_loader = DataLoader(
    train_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    num_workers=8,
    # collate_fn=partial(conditional_cutmix_collate, alpha=0.7, num_classes=num_classes, cutmix_prob=0.3)
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    num_workers=8,
)


model = convnext_base(weights=ConvNeXt_Base_Weights.DEFAULT)
model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
model = model.to(device)

class MultiFocalLoss(nn.Module):
    def __init__(self, gammas=(1.0, 2.0), weights=(0.6, 0.4), reduction='mean'):
        super(MultiFocalLoss, self).__init__()
        self.gammas = gammas
        self.weights = weights
        self.reduction = reduction

    def forward(self, inputs, targets):
        total_loss = 0
        for gamma, weight in zip(self.gammas, self.weights):
            ce_loss = F.cross_entropy(inputs, targets, reduction='none')
            pt = torch.exp(-ce_loss)
            focal_loss = weight * (1 - pt) ** gamma * ce_loss
            if self.reduction == 'mean':
                total_loss += focal_loss.mean()
            elif self.reduction == 'sum':
                total_loss += focal_loss.sum()
            else:
                total_loss += focal_loss  # 'none'
        return total_loss


criterion = MultiFocalLoss(
    gammas=(1.0, 2.0),  # gamma 2종류
    weights=(0.6, 0.4), # 가중 평균

)

# ✅ 옵티마이저, 손실함수, 스케줄러
#riterion = FocalLoss(alpha=1, gamma=1.0) # gamma=1 유지
optimizer = optim.AdamW(model.parameters(), lr=CFG['LEARNING_RATE'], weight_decay=CFG['WEIGHT_DECAY'])
scaler = GradScaler()

total_steps = len(train_loader) * CFG['EPOCHS']
warmup_steps = int(total_steps * 0.02)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# ✅ EarlyStopping 준비
best_loss = float('inf')
best_acc = 0.0
patience, counter = 5, 0

# ✅ 학습 루프
for epoch in range(CFG['EPOCHS']):
    train_loss = 0
    model.train()

    for images, labels in tqdm(train_loader, desc=f"[Train Epoch {epoch+1}]"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)


    # ✅ Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            with autocast(device_type='cuda'):
                outputs = model(images) / CFG['TEMPERTURE']
                loss = log_loss_manual(outputs, labels)
                # loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = correct / total
    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # ✅ Best Model 저장
    if val_loss < best_loss or (val_loss - best_loss <= 0.05 and val_acc > best_acc):
        best_loss = val_loss
        best_acc = val_acc
        torch.save(model.state_dict(), f'pt/convnext_ver2_holdout_best.pth')
        print(f"✅ Best model saved at epoch {epoch+1}")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"⏹️ Early stopping at epoch {epoch+1}")
            break

[Train Epoch 1]: 100%|██████████| 1001/1001 [42:23<00:00,  2.54s/it]


[Epoch 1] Train Loss: 4.1437 | Val Loss: 0.9966 | Val Acc: 0.8476
✅ Best model saved at epoch 1


[Train Epoch 2]: 100%|██████████| 1001/1001 [04:07<00:00,  4.04it/s]


[Epoch 2] Train Loss: 0.5540 | Val Loss: 0.2718 | Val Acc: 0.9395
✅ Best model saved at epoch 2


[Train Epoch 3]: 100%|██████████| 1001/1001 [04:12<00:00,  3.96it/s]


...

[Epoch 28] Train Loss: 0.0169 | Val Loss: 0.0403 | Val Acc: 0.9869


[Train Epoch 29]: 100%|██████████| 1001/1001 [04:08<00:00,  4.02it/s]


[Epoch 29] Train Loss: 0.0159 | Val Loss: 0.0397 | Val Acc: 0.9869
✅ Best model saved at epoch 29


[Train Epoch 30]: 100%|██████████| 1001/1001 [04:13<00:00,  3.94it/s]


[Epoch 30] Train Loss: 0.0169 | Val Loss: 0.0407 | Val Acc: 0.9869


In [ ]:
# ✅ 모델 로드
model.load_state_dict(torch.load('pt/convnext_ver2_holdout_best.pth'))
model.eval()
model.to(device)

# ✅ 검증 데이터셋 다시 구성 (transform 동일하게)
val_dataset = Subset(datasets.ImageFolder(train_root, transform=val_transform), val_idx)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

# ✅ 클래스 이름 및 파일 경로 추출
full_dataset = datasets.ImageFolder(train_root)  # 파일 경로 정보용
class_names = full_dataset.classes
val_samples = [full_dataset.samples[i] for i in val_idx]  # (path, label)

# ✅ 오답 샘플 저장용 리스트
misclassified = []

with torch.no_grad():
    for i, (images, labels) in enumerate(tqdm(val_loader, desc="🔍 Validating")):
        images, labels = images.to(device), labels.to(device)
        with autocast(device_type='cuda'):
            outputs = model(images) / CFG['TEMPERTURE']
            probs = F.softmax(outputs, dim=1)  # ✅ 확률로 변환
        preds = torch.argmax(outputs, dim=1)

        batch_start = i * CFG['BATCH_SIZE']
        for j in range(len(labels)):
            if preds[j] != labels[j]:
                sample_idx = batch_start + j
                img_path, true_label = val_samples[sample_idx]
                pred_label = preds[j].item()
                misclassified.append({
                    'img_path': img_path,
                    'true_label': class_names[true_label],
                    'pred_label': class_names[pred_label],
                    'true_prob': probs[j][true_label].item(),  # ✅ 정답 클래스 확률
                    'pred_prob': probs[j][pred_label].item(),  # ✅ 예측 클래스 확률
                })

# ✅ Pandas DataFrame으로 정리
df_misclassified = pd.DataFrame(misclassified)
df_misclassified.to_csv("val_misclassified_samples_2.csv", index=False, encoding="utf-8-sig")
print(f"총 오답 수: {len(df_misclassified)}")
df_misclassified

🔍 Validating: 100%|██████████| 31/31 [00:04<00:00,  6.95it/s]


총 오답 수: 14


,img_path,true_label,pred_label,true_prob,pred_prob
0,./data/train/5008_2세대_2021_2024/5008_2세대_2...,5008_2세대_2021_2024,5008_2세대_2018_2019,0.160383,0.833806
1,./data/train/올_뉴_K7_하이브리드_2017_2019/오...,올_뉴_K7_하이브리드_2017_2019,올_뉴_K7_2016_2019,0.425582,0.572683
2,./data/train/레인지로버_스포츠_2세대_2013_201...,레인지로버_스포츠_2세대_2013_2017,레인지로버_스포츠_2세대_2018_2022,0.018669,0.980259
3,./data/train/X4_G02_2019_2021/X4_G02_2019_2021...,X4_G02_2019_2021,X3_G01_2018_2021,0.279380,0.719016
4,./data/train/티볼리_아머_2018_2019/티볼리_아...,티볼리_아머_2018_2019,티볼리_2015_2018,0.111207,0.888489
5,./data/train/뉴_A6_2015_2018/뉴_A6_2015_2018_0...,뉴_A6_2015_2018,뉴_A6_2012_2014,0.079535,0.917368
6,./data/train/트레일블레이저_2023/트레일블ᄅ...,트레일블레이저_2023,트레일블레이저_2021_2022,0.356372,0.635302
7,./data/train/3시리즈_F30_2013_2018/3시리즈_F30...,3시리즈_F30_2013_2018,M4_F82_2015_2020,0.055974,0.875574
8,./data/train/올_뉴_K7_2016_2019/올_뉴_K7_201...,올_뉴_K7_2016_2019,올_뉴_K7_하이브리드_2017_2019,0.233601,0.765947
9,./data/train/K7_프리미어_2020_2021/K7_프리미어...,K7_프리미어_2020_2021,K7_프리미어_하이브리드_2020_2021,0.474449,0.525167


# Inference

In [ ]:
import torch
import torch.nn.functional as F
from torchvision.models import convnext_base
from torchvision.models import ConvNeXt_Base_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import unicodedata
from scipy.special import logsumexp, softmax

In [ ]:
# ✅ TestDataset 정의 추가
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_names = sorted(os.listdir(img_dir))
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.img_names[idx]

In [ ]:
# ✅ TTA Transform 조합 정의
tta_transforms = [
    test_transform,  # 원본
    transforms.Compose([
        transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
        transforms.RandomHorizontalFlip(p=1.0),  # 좌우 반전
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
        transforms.RandomRotation(5),  # 회전
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
]

In [ ]:
# ✅ 테스트셋 로드
test_dataset = TestDataset(test_root, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

In [ ]:
# ✅ 준비
num_classes = len(class_names)
SEEDS = [42]  # 사용할 시드 리스트
seed_logits_list = []

for seed in SEEDS:
    print(f"📦 Loading model for seed {seed}...")

    model = convnext_base(weights=ConvNeXt_Base_Weights.DEFAULT)
    model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
    model.load_state_dict(torch.load(f'./pt/convnext_ver2_holdout_best.pth', map_location=device))
    model = model.to(device)
    model.eval()

    # ✅ TTA별 raw logits 수집
    tta_logits_list = []
    for tta_transform in tta_transforms:
        tta_dataset = TestDataset(test_root, transform=tta_transform)
        tta_loader = DataLoader(tta_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

        all_logits = []
        with torch.no_grad():
            for images, _ in tqdm(tta_loader, desc=f"[Seed {seed} | TTA]"):
                images = images.to(device)
                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(images)  # Raw logits
                all_logits.append(outputs.cpu().numpy())

        logits = np.concatenate(all_logits, axis=0)  # [N, C]
        tta_logits_list.append(logits)

    # ✅ TTA 기하 평균: log-sum-exp
    tta_logits_stack = np.stack(tta_logits_list, axis=0)  # [T, N, C]
    logsumexp_logits = logsumexp(tta_logits_stack, axis=0) - np.log(len(tta_logits_list))  # [N, C]
    seed_logits_list.append(logsumexp_logits)

# ✅ Seed별 기하 평균: log-sum-exp
seed_logits_stack = np.stack(seed_logits_list, axis=0)  # [S, N, C]
final_logits = logsumexp(seed_logits_stack, axis=0) - np.log(len(seed_logits_list))  # [N, C]

# ✅ Temperature Scaling 적용
scaled_logits = final_logits / CFG['TEMPERTURE']
final_probs = torch.softmax(torch.tensor(scaled_logits), dim=1).numpy()  # [N, C]

# ✅ 결과 정리
results = []
for prob in final_probs:
    result = {class_names[i]: prob[i].item() for i in range(len(class_names))}
    results.append(result)

pred = pd.DataFrame(results)

📦 Loading model for seed 42...


[Seed 42 | TTA]: 100%|██████████| 259/259 [04:30<00:00,  1.04s/it]
[Seed 42 | TTA]: 100%|██████████| 259/259 [01:12<00:00,  3.59it/s]
[Seed 42 | TTA]: 100%|██████████| 259/259 [00:25<00:00,  9.99it/s]


In [ ]:
pred.columns = [unicodedata.normalize("NFC", col) for col in pred.columns]
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8258 entries, 0 to 8257
Columns: 396 entries, 1시리즈_F20_2013_2015 to 프리우스_C_2018_2020
dtypes: float64(396)
memory usage: 24.9 MB


# Submission

In [ ]:
# 제출 양식 불러오기
submission = pd.read_csv('./data/sample_submission.csv', encoding='utf-8-sig')
class_columns = submission.columns[1:]

# 누락된 클래스 컬럼 확인
missing_cols = [col for col in class_columns if col not in pred.columns]
for col in missing_cols:
    pred[col] = 0

# 클래스 컬럼 순서 submission과 일치하도록 정렬
pred = pred[class_columns]

# 결과 반영
submission[class_columns] = pred.values
submission.to_csv('convnext_base_meg.csv', index=False, encoding='utf-8-sig')